In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\deskone\Desktop\wargame\songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

In [6]:
df['text'][0]

"With the breaking of the dawn  \nI will peek beyond the glass  \nThere' s an endless stretch of blue below  \nAn ocean cold and vast  \nI am traveling again  \nBetween the heavens and the clouds  \nReaching out, turning a vision  \nInto a future, strong and proud  \n  \nBut I say, oh, there' s nothing more I desire  \nThan to spend my life with you  \nSetting me gently on fire.  \n  \nWe' re a very special two  \nOnly you can understand  \nWalking parallel, on separate paths,  \nBut always holding hands.  \nAs we lay our yellow bricks  \nOn the road to something better  \nCan we find a point of meeting  \nAnd continue on together?  \n  \nAnd I say, oh, - there' s nothing more I desire  \nThan to spend my life with you  \nSetting me gently on fire.\n\n"

In [7]:
 df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

0       with the breaking of the dawn   i will peek be...
1       me i'm just a poor boy, from across the far so...
2       i come to you with strange fire   i make an of...
3       believe me, that's just a lie   that's what sh...
4       park the car at the side of the road   you sho...
                              ...                        
4995    sophistophone, aristocrats   distinguished sta...
4996    i'm in love with the sunshine   i'm in love wi...
4997    oh you are lying (lying), you said it was fore...
4998    introduction   god bless america, the land tha...
4999    i once knew a girl   i can't remember her name...
Name: text, Length: 5000, dtype: object

In [8]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [9]:
df['text'][0]

"with the breaking of the dawn   i will peek beyond the glass   there' s an endless stretch of blue below   an ocean cold and vast   i am traveling again   between the heavens and the clouds   reaching out, turning a vision   into a future, strong and proud      but i say, oh, there' s nothing more i desire   than to spend my life with you   setting me gently on fire.      we' re a very special two   only you can understand   walking parallel, on separate paths,   but always holding hands.   as we lay our yellow bricks   on the road to something better   can we find a point of meeting   and continue on together?      and i say, oh, - there' s nothing more i desire   than to spend my life with you   setting me gently on fire.  "

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens]
    return " ".join(stemming)

In [13]:
tokenization('this is my loving loved')

'thi is my love love'

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfid = TfidfVectorizer(stop_words='english')
matrix = tfid.fit_transform(df['text'])

In [18]:
matrix

<5000x17481 sparse matrix of type '<class 'numpy.float64'>'
	with 277737 stored elements in Compressed Sparse Row format>

In [19]:
matrix.shape

(5000, 17481)

In [20]:
similarity = cosine_similarity(matrix)

In [21]:
similarity

array([[1.        , 0.03265957, 0.02342463, ..., 0.01842125, 0.02403999,
        0.0050768 ],
       [0.03265957, 1.        , 0.01135866, ..., 0.01615333, 0.03451872,
        0.03251951],
       [0.02342463, 0.01135866, 1.        , ..., 0.03813211, 0.04401934,
        0.05608708],
       ...,
       [0.01842125, 0.01615333, 0.03813211, ..., 1.        , 0.00405421,
        0.02342769],
       [0.02403999, 0.03451872, 0.04401934, ..., 0.00405421, 1.        ,
        0.0131174 ],
       [0.0050768 , 0.03251951, 0.05608708, ..., 0.02342769, 0.0131174 ,
        1.        ]])

In [22]:
df['song'][0]

'Desire'

In [23]:
df[df['song']=='Desire']

,artist,song,text
0,Noa,Desire,with the break of the dawn i will peek beyond ...
2533,Slayer,Desire,no one could ever doubt my raptur as you too w...


In [32]:
def recommendation(song):
    idx = df[df['song'] == song].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=False,key=lambda x:x[1])
    
    songs = []
    for i in distances[1:21]:
        songs.append(df.iloc[i[0]].song)
        
    return songs

In [34]:
recommendation('The Way It Was')

['Everything Grows',
 'Ride',
 'Krs-One',
 'Junk',
 'Kota',
 'Marigold',
 'Piggies',
 'Mann Mein Kisi Ki Preet Basa Le',
 'Aku Cinta Kau Dan Dia',
 'Five Magics',
 'Tonight',
 'Mississippi River',
 'Phenomena',
 'Mundzumundbeatmung',
 'Pesawat Tempurku',
 'Doa Pengorbral Dosa',
 'Once In A Lifetime',
 "Let 'em In",
 'River Jordan',
 'Sharks']

In [36]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))